In [1]:
pip install fastcluster

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.0/194.0 kB 4.1 MB/s eta 0:00:00a 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [2]:
import os
import sys
import time
# import hdbscan
import warnings
import numpy as np
import pandas as pd
# import seaborn as sns
# import networkx as nx
# import community as community_louvain  # Louvain algorithm
import matplotlib.pyplot as plt
import matplotlib.patches as patches
# import missingno as mno
# import racplusplus
# from decimal import Decimal
# from miceforest import ImputationKernel
# from sklearn.experimental import enable_iterative_imputer
# from sklearn.linear_model import LinearRegression, BayesianRidge
from sklearn.neighbors import KNeighborsRegressor, kneighbors_graph, NearestNeighbors
# from sklearn.tree import DecisionTreeRegressor
from sklearn.impute import SimpleImputer #, IterativeImputer, KNNImputer
from sklearn.preprocessing import StandardScaler, LabelEncoder
# from sklearn.neural_network import MLPRegressor
from sklearn.cluster import AgglomerativeClustering, KMeans, Birch
from sklearn.semi_supervised import LabelPropagation
from sklearn.datasets import make_blobs
from sklearn import datasets
from sklearn.decomposition import PCA
from sklearn.metrics.pairwise import euclidean_distances
from sklearn.manifold import TSNE
from scipy.cluster.hierarchy import linkage, fcluster, dendrogram
from scipy.spatial.distance import pdist, squareform, euclidean
from scipy.sparse import lil_matrix
from fastcluster import linkage as flinkage
from collections import Counter

In [3]:
data = pd.read_csv("/kaggle/input/private-blood-count/labelled_first_all_dropped.csv", index_col = 0)
data.reset_index(drop = True, inplace = True)

/tmp/ipykernel_30/824357630.py:1: DtypeWarning: Columns (59,60) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv("/kaggle/input/private-blood-count/labelled_first_all_dropped.csv", index_col = 0)


In [4]:
labels = data.iloc[:, 60]
first = data.iloc[:, :58]

In [5]:
first

,WBC(10^3/uL),RBC(10^6/uL),HGB(g/dL),HCT(%),MCV(fL),MCH(pg),MCHC(g/dL),PLT(10^3/uL),NRBC#(10^3/uL),NRBC%(%),...,[MO-Z(ch)],[NE-WX],[NE-WY],[NE-WZ],[LY-WX],[LY-WY],[LY-WZ],[MO-WX],[MO-WY],[MO-WZ]
0,3.02,2.42,5.6,17.8,73.6,23.1,31.5,87.0,0.14,4.6,...,95.6,357.0,580.0,657.0,772.0,602.0,706.0,368.0,837.0,743.0
1,17.42,5.15,15.1,44.4,86.2,29.3,34.0,552.0,1.17,6.7,...,104.3,332.0,725.0,759.0,725.0,940.0,741.0,366.0,913.0,786.0
2,6.87,4.44,12.1,36.2,81.5,27.3,33.4,242.0,0.43,6.3,...,99.0,349.0,708.0,727.0,735.0,756.0,697.0,316.0,762.0,767.0
3,7.04,4.50,12.0,36.5,81.1,26.7,32.9,227.0,0.36,5.1,...,100.1,365.0,667.0,712.0,754.0,708.0,653.0,368.0,753.0,709.0
4,2.86,2.31,5.6,17.1,74.0,24.2,32.7,88.0,0.17,5.9,...,100.6,318.0,622.0,795.0,771.0,718.0,738.0,374.0,796.0,686.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
79289,288.66,2.85,6.9,22.7,79.6,24.2,30.4,657.0,0.61,0.2,...,70.9,602.0,3186.0,931.0,1182.0,3451.0,1041.0,545.0,1058.0,1129.0
79290,96.39,3.61,10.5,33.5,92.8,29.1,31.3,754.0,0.23,0.2,...,63.3,423.0,2670.0,885.0,602.0,1376.0,925.0,444.0,1121.0,948.0
79291,90.98,2.67,8.4,28.3,106.0,31.5,29.7,327.0,3.24,3.6,...,64.7,578.0,3030.0,1032.0,651.0,1531.0,880.0,322.0,1411.0,788.0
79292,29.27,5.15,14.5,46.4,90.1,28.2,31.3,203.0,0.01,0.0,...,60.7,304.0,619.0,653.0,501.0,1007.0,576.0,245.0,643.0,643.0


In [6]:
scaler = StandardScaler()
cols = first.columns
first = pd.DataFrame(scaler.fit_transform(first), columns = cols)

In [7]:
first

,WBC(10^3/uL),RBC(10^6/uL),HGB(g/dL),HCT(%),MCV(fL),MCH(pg),MCHC(g/dL),PLT(10^3/uL),NRBC#(10^3/uL),NRBC%(%),...,[MO-Z(ch)],[NE-WX],[NE-WY],[NE-WZ],[LY-WX],[LY-WY],[LY-WZ],[MO-WX],[MO-WY],[MO-WZ]
0,-0.508105,-1.319162,-1.738721,-1.897984,-2.023744,-1.293119,0.443654,-1.126483,0.128086,0.738325,...,6.583189,0.947356,-0.501936,-0.420921,2.295455,-1.860041,0.975832,2.408648,0.928034,1.316543
1,0.364286,1.586208,1.988614,1.452782,-0.526756,0.466194,1.547825,2.057950,1.540464,1.104532,...,8.370568,0.291602,0.303545,0.963143,1.822178,-0.147938,1.300254,2.361072,1.447050,1.801796
2,-0.274862,0.830599,0.811561,0.419839,-1.085156,-0.101326,1.282824,-0.065005,0.525746,1.034779,...,7.281705,0.737515,0.209110,0.528927,1.922875,-1.079970,0.892409,1.171662,0.415847,1.587382
3,-0.264563,0.894453,0.772325,0.457630,-1.132680,-0.271582,1.061989,-0.167729,0.429759,0.825517,...,7.507695,1.157198,-0.018647,0.325388,2.114200,-1.323109,0.484563,2.408648,0.354385,0.932854
4,-0.517798,-1.436228,-1.738721,-1.986162,-1.976221,-0.980983,0.973656,-1.119635,0.169223,0.965025,...,7.610418,-0.075620,-0.268624,1.451635,2.285386,-1.272455,1.272446,2.551378,0.648039,0.673300
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
79289,16.796744,-0.861540,-1.228665,-1.280738,-1.310893,-0.980983,-0.042182,2.777016,0.772569,-0.028968,...,1.508674,7.373748,13.974506,3.297053,6.424042,12.571268,4.081018,6.619161,2.437278,5.672538
79290,5.148504,-0.052719,0.183799,0.079724,0.257381,0.409442,0.355320,3.441295,0.251498,-0.028968,...,-0.052715,2.678548,11.108104,2.672868,0.583602,2.060574,3.005789,4.216552,2.867515,3.629960
79291,4.820751,-1.053103,-0.640138,-0.575313,1.825654,1.090466,-0.351350,0.517095,4.378932,0.563940,...,0.234909,6.744224,13.107919,4.667547,1.077019,2.845711,2.588675,1.314391,4.847971,1.824366
79292,1.082191,1.586208,1.753203,1.704719,-0.063402,0.154058,0.355320,-0.332087,-0.050175,-0.063845,...,-0.586874,-0.442843,-0.285289,-0.475198,-0.433440,0.191444,-0.229166,-0.517301,-0.396823,0.188046


# FastCluster

In [ ]:
labels

# Ward

In [11]:
labelled_points = data.Diagnosis[-54:].copy()
unique_labels = pd.DataFrame(sorted(list(labelled_points.unique())), columns = ['labels'])
unique_labels['frequencies'] = labelled_points.value_counts()[unique_labels.labels].values
unique_labels

,labels,frequencies
0,Acute leukemia,5
1,Acute leukemia monocytic,1
2,Acute lymphoblastic leukemia,11
3,Acute myeloid leukemia,5
4,Acute myeloid leukemia morphologically monocytic,1
5,Acute myeloid leukemia non M3,1
6,Acute myelomonocytic leukemia,3
7,Aplastic anemia,1
8,Atypical chronic lymphocytic leukemia,1
9,Chronic lymphocytic leukemia,3


In [ ]:
# tic = time.time()
# Z_ward = flinkage(first, method = 'ward')
# toc = time.time()
# print(f"{toc - tic} seconds")
# # Z_ward = np.loadtxt('/kaggle/input/private-blood-count/Z_ward.txt', dtype = 'float')
# Z_ward

In [8]:
Z_ward = np.loadtxt('/kaggle/input/private-blood-count/Z_ward.txt', dtype = 'float')
Z_ward

array([[4.18470000e+04, 7.92920000e+04, 0.00000000e+00, 2.00000000e+00],
       [5.40130000e+04, 5.40150000e+04, 5.93776000e-01, 2.00000000e+00],
       [4.04700000e+03, 4.04800000e+03, 6.69828000e-01, 2.00000000e+00],
       ...,
       [1.58581000e+05, 1.58583000e+05, 6.81585607e+02, 7.84580000e+04],
       [1.58540000e+05, 1.58584000e+05, 8.47228865e+02, 7.92020000e+04],
       [1.58578000e+05, 1.58585000e+05, 1.02896000e+03, 7.92940000e+04]])

In [ ]:
np.savetxt('Z_ward.txt', Z_ward, fmt = "%f")

In [ ]:
# (Z_ward == Z_ward_import).sum(axis = 0)

In [9]:
len(Z_ward)

79293

In [ ]:
# assume that the clusters are distinct. find an arbitrary cut.
cluster_labels = fcluster(Z_ward, t = 18, criterion = 'maxclust')
cluster_labels = pd.Series(cluster_labels)

In [ ]:
cluster_labels.value_counts()

In [ ]:
# tic = time.time()
# tsne = TSNE(n_components=2, perplexity=30, learning_rate=200, random_state=42)
# data_2d_tsne = tsne.fit_transform(first)
# toc = time.time()

In [ ]:
# plt.figure(figsize=(20, 20))
# plt.scatter(data_2d_tsne[:, 0], data_2d_tsne[:, 1], s=1)
# plt.title('t-SNE Visualization')
# plt.show()

In [ ]:
data_2d_tsne = np.loadtxt('/kaggle/input/private-blood-count/data_2d_tsne.txt', dtype = 'float')
data_2d_tsne

In [ ]:
mask = data['Diagnosis'].str.contains('chronic myeloid leukemia', case = False)
flag_mask = mask.copy()
flag_mask[flag_mask.isna()] = 0
mask[mask.isna()] = False
mask = mask.astype(bool)
flag_mask[mask] = 1
~mask[-54:]
# ~mask
flag_mask[-54:][~mask[-54:]] = 2
flag_mask
mask
# flag_mask[~flag_mask] = 2
# flag_mask

# subset = data[mask]
# steps: wherever mask is True, replace it with 1
# wherever mask is False and the index is in the last 54 rows (i.e. labelled rows), replace it with 0
# otherwise, replace it with 2. do this using a duplicate mask

In [ ]:
plt.figure(figsize=(20, 20))
plt.scatter(data_2d_tsne[:, 0], data_2d_tsne[:, 1], s=1, c=cluster_labels, cmap='viridis')
circle_count_1 = 0
circle_count_2 = mask.sum()
avg_of_points_x = 0
avg_of_points_y = 0
indices_cml_close = []
for i in range(len(data_2d_tsne)):
    if flag_mask[i] == 1:
        if circle_count_1 != 8:
            avg_of_points_x += data_2d_tsne[i, 0]
            avg_of_points_y += data_2d_tsne[i, 1]
            indices_cml_close.append(i)
        circle = patches.Circle((data_2d_tsne[i, 0], data_2d_tsne[i, 1]), radius=0.1, color='black', fill=False)
        plt.gca().add_patch(circle)
        plt.text(data_2d_tsne[i, 0], data_2d_tsne[i, 1] + 0.18, str(circle_count_1), fontsize=12, ha='center', va='center')
        circle_count_1 += 1
    elif flag_mask[i] == 2:
        if circle_count_2 == 51:
            print(i)
        circle = patches.Circle((data_2d_tsne[i, 0], data_2d_tsne[i, 1]), radius=0.1, color='blue', fill=False)
        plt.gca().add_patch(circle)
        plt.text(data_2d_tsne[i, 0], data_2d_tsne[i, 1] + 0.18, str(circle_count_2), fontsize=12, ha='center', va='center')
        circle_count_2 += 1
print(data.loc[i, 'Diagnosis'])
avg_of_points_x /= circle_count_1 - 1
avg_of_points_y /= circle_count_1 - 1
print(indices_cml_close)
circle = patches.Circle((avg_of_points_x, avg_of_points_y), radius = 4.6, color = "red", fill = False)
plt.gca().add_patch(circle)
plt.text(avg_of_points_x + 5, avg_of_points_y + 5, "CML POINTS", fontsize = 25, ha = 'center', va = 'center')
plt.title('t-SNE Visualization')
plt.savefig('t-SNE Visualization v2.png')
plt.show()

In [ ]:
def plot_helper(string, flag_mask, plot_others = True, cluster_labels = cluster_labels, xlim = None, ylim = None, big_plot = True, t = None):
    if big_plot:
        plt.figure(figsize=(20, 20))
        plt.scatter(data_2d_tsne[:, 0], data_2d_tsne[:, 1], s=1, c=cluster_labels, cmap='viridis')
        circle_count_1 = 0
        circle_count_2 = mask.sum()
        avg_of_points_x = 0
        avg_of_points_y = 0
        indices_cml_close = []
        d1 = {}
        d2 = {}
        for i in range(len(data_2d_tsne)):
            if flag_mask[i] == 1:
#                 if circle_count_1 != 8:
#                     avg_of_points_x += data_2d_tsne[i, 0]
#                     avg_of_points_y += data_2d_tsne[i, 1]
#                     indices_cml_close.append(i)
                circle = patches.Circle((data_2d_tsne[i, 0], data_2d_tsne[i, 1]), radius=0.1, color='black', fill=False)
                plt.gca().add_patch(circle)
                plt.text(data_2d_tsne[i, 0], data_2d_tsne[i, 1] + 0.6, str(circle_count_1), fontsize=12, ha='center', va='center')
                d1[circle_count_1] = data.loc[i, 'Diagnosis']
                circle_count_1 += 1
            if plot_others == True:
                if flag_mask[i] == 2:
                    circle = patches.Circle((data_2d_tsne[i, 0], data_2d_tsne[i, 1]), radius=0.1, color='blue', fill=False)
                    plt.gca().add_patch(circle)
                    plt.text(data_2d_tsne[i, 0], data_2d_tsne[i, 1] + 0.6, str(circle_count_2), fontsize=12, ha='center', va='center')
                    d2[circle_count_2] = data.loc[i, 'Diagnosis']
                    circle_count_2 += 1
#         avg_of_points_x /= circle_count_1 - 1
#         avg_of_points_y /= circle_count_1 - 1
#         circle = patches.Circle((avg_of_points_x, avg_of_points_y), radius = 4.6, color = "red", fill = False)
#         plt.gca().add_patch(circle)
#         plt.text(avg_of_points_x - 5, avg_of_points_y + 5, f"{string} POINTS", fontsize = 15, ha = 'center', va = 'center')
        plt.title(f't-SNE Visualization of {string}')
        plt.savefig(f"t-SNE Visualization std iter {string}.png", bbox_inches = 'tight')
        plt.show()
    
    # Zoomed-in plot if xlim and ylim are provided
    if xlim and ylim:
        plt.figure(figsize = (20, 20))
        plt.scatter(data_2d_tsne[:, 0], data_2d_tsne[:, 1], s=6.5, c=cluster_labels, cmap='viridis')
        plt.xlim(xlim)
        plt.ylim(ylim)
        circle_count_1 = 0
        circle_count_2 = mask.sum()
        avg_of_points_x = 0
        avg_of_points_y = 0
        for i in range(len(data_2d_tsne)):
            if flag_mask[i] == 1:
                if circle_count_1 != 8:
                    avg_of_points_x += data_2d_tsne[i, 0]
                    avg_of_points_y += data_2d_tsne[i, 1]
                circle = patches.Circle((data_2d_tsne[i, 0], data_2d_tsne[i, 1]), radius=0.1, color='black', fill=False)
                plt.gca().add_patch(circle)
                circle_count_1 += 1
            elif flag_mask[i] == 2:
                if plot_others == True:
                    circle = patches.Circle((data_2d_tsne[i, 0], data_2d_tsne[i, 1]), radius=0.1, color='blue', fill=False)
                    plt.gca().add_patch(circle)
                    circle_count_2 += 1
        avg_of_points_x /= circle_count_1 - 1
        avg_of_points_y /= circle_count_1 - 1
        circle = patches.Circle((avg_of_points_x, avg_of_points_y), radius = 4.6, color = "red", fill = False)
        plt.gca().add_patch(circle)
        if t:
            plt.savefig(f"t-SNE Visualization std iter {t}.png", bbox_inches = 'tight')
        plt.show()
    if big_plot == True:
        return d1, d2
string = 'Chronic myeloid leukemia chronic phase'
plot_helper(string, flag_mask)
d1, d2 = plot_helper(string, flag_mask, plot_others = True, cluster_labels = cluster_labels, xlim = (30, 50), ylim = (-10, 10))

In [ ]:
d1

In [ ]:
a = np.array([26, 38, 17, 14, 25, 51, 23, 28, 42, 19, 47, 33, 43, 48, 21])
a.sort()
s2 = pd.Series(d2)
b = np.array(s2.index)
c = np.setdiff1d(b, a)
s2[a].value_counts()

In [ ]:
s2[c].value_counts()

I have 2 ways of approaching this issue:
* I either loop through every single disease and find the mask for each, then plot it accordingly
* I use it point by point

In [ ]:
nunq = data.loc[-54:, 'Diagnosis'].value_counts()

In [ ]:
last_54 = data.iloc[-54:, ].Diagnosis.unique()
last_54.sort()
for l in last_54:
    print(l, nunq[l])

In [ ]:
# def mask_generator(string):
#     mask = data['Diagnosis'].str.contains(string, case = False)
#     flag_mask = mask.copy()
#     flag_mask[flag_mask.isna()] = 0
#     mask[mask.isna()] = False
#     mask = mask.astype(bool)
#     flag_mask[mask] = 1
#     flag_mask[-54:][~mask[-54:]] = 2
#     plot_helper(string, mask, plot_others = False, cluster_labels = ~flag_mask)
# mask_generator('Acute leukemia')

def mask_generator(string):
    with warnings.catch_warnings():
        warnings.simplefilter(action='ignore', category=FutureWarning)
#         mask = data['Diagnosis'].str.contains(string, case=False)
        mask = data['Diagnosis'] == string
        flag_mask = mask.copy()
        flag_mask.loc[flag_mask.isna()] = 0  # Use .loc for assignment
        mask.loc[mask.isna()] = False        # Use .loc for assignment
        mask = mask.astype(bool)
        flag_mask[mask] = 1
        flag_mask[-54:][~mask[-54:]] = 2
    plot_helper(string = string, flag_mask = mask, plot_others=False, cluster_labels=~mask)

In [ ]:
def set_mask_generator(string_set):
    with warnings.catch_warnings():
        warnings.simplefilter(action='ignore', category=FutureWarning)
        big_mask = np.array([False] * len(data))
        for string in string_set:
#         mask = data['Diagnosis'].str.contains(string, case=False)
            mask = data['Diagnosis'] == string
            flag_mask = mask.copy()
            flag_mask.loc[flag_mask.isna()] = 0  # Use .loc for assignment
            mask.loc[mask.isna()] = False        # Use .loc for assignment
            mask = mask.astype(bool)
            flag_mask[mask] = 1
            flag_mask[-54:][~mask[-54:]] = 2
            big_mask[mask] = True
    plot_helper(string = string, flag_mask = big_mask, plot_others=False, cluster_labels=~big_mask)

In [ ]:
string_set = ['Acute leukemia', 'Acute leukemia monocytic', 'Acute lymphoblastic leukemia', 'Acute myeloid leukemia', 'Acute myeloid leukemia morphologically monocytic', 'Acute myeloid leukemia non M3', 'Acute myelomonocytic leukemia', 'Atypical chronic lymphocytic leukemia', 'Chronic lymphocytic leukemia']

In [ ]:
set_mask_generator(string_set)

In [ ]:
# nunq

In [ ]:
# for n in nunq.index:
#     print(n, nunq[n])
#     mask_generator(n)

In [ ]:
# a = np.array([26, 38, 17, 14, 25, 51, 23, 28, 42, 19, 47, 33, 43, 48, 21, 24, 40])
# a.sort()
# s2 = pd.Series(d2)
# b = np.array(s2.index)
# c = np.setdiff1d(b, a)
# s2[a].value_counts()

In [ ]:
# s2[c].value_counts()

In [ ]:
# cluster_mask = pd.Series([False] * len(first))
# for c in cluster_labels_set:
#     cluster_mask[clusters == c] = True
# plot_helper(~cluster_mask, (30, 50), (-10, 10))
#     # find the indices where the cluster is in the set
#     # set those indices to true, use this to create a new mask.

In [ ]:
# threshold = 198.95
# round(threshold, ndigits = 1)

In [ ]:
# def aux_plot(cluster_labels):
#     plt.figure(figsize=(20, 20))
#     plt.scatter(data_2d_tsne[:, 0], data_2d_tsne[:, 1], s=5, c=cluster_labels, cmap='viridis')
# #     circle_count_1 = 0
# #     circle_count_2 = mask.sum()
# #     avg_of_points_x = 0
# #     avg_of_points_y = 0
# #     indices_cml_close = []
# #     for i in range(len(data_2d_tsne)):
# #         if flag_mask[i] == 1:
# #             if circle_count_1 != 8:
# #                 avg_of_points_x += data_2d_tsne[i, 0]
# #                 avg_of_points_y += data_2d_tsne[i, 1]
# #                 indices_cml_close.append(i)
# #         if cluster_labels[i] == True:
# #             circle = patches.Circle((data_2d_tsne[i, 0], data_2d_tsne[i, 1]), radius=0.1, color='black', fill=False)
# #             plt.gca().add_patch(circle)
# #             plt.text(data_2d_tsne[i, 0], data_2d_tsne[i, 1] + 0.18, str(circle_count_1), fontsize=12, ha='center', va='center')
# #                 circle_count_1 += 1
# #         elif flag_mask[i] == 2:
# #             circle = patches.Circle((data_2d_tsne[i, 0], data_2d_tsne[i, 1]), radius=0.1, color='blue', fill=False)
# #             plt.gca().add_patch(circle)
# #             plt.text(data_2d_tsne[i, 0], data_2d_tsne[i, 1] + 0.18, str(circle_count_2), fontsize=12, ha='center', va='center')
# #             circle_count_2 += 1
# #     avg_of_points_x /= circle_count_1 - 1
# #     avg_of_points_y /= circle_count_1 - 1
# #     circle = patches.Circle((avg_of_points_x, avg_of_points_y), radius = 4.6, color = "red", fill = False)
# #     plt.gca().add_patch(circle)
# #     plt.text(avg_of_points_x + 5, avg_of_points_y + 5, "CML POINTS", fontsize = 25, ha = 'center', va = 'center')
#     plt.title('t-SNE Visualization')
#     plt.show()

In [ ]:
# k = 5000
# # Iterate over different threshold levels Z_ward[-1, 2]
# for threshold in np.arange(201.3, 201.5, 0.1):  # or fine-tune based on your dendrogram structure
#     threshold = round(threshold, ndigits = 1)
#     clusters = fcluster(Z_ward, t=threshold, criterion='distance')
#     cluster_labels_set = set(clusters[i] for i in indices_cml_close)
#     cluster_sizes = {label: (clusters == label).sum() for label in cluster_labels_set}  # Dictionary with cluster sizes
#     print(f'{threshold}: {cluster_sizes}')
#     cluster_mask = pd.Series([False] * len(first))
#     for c in cluster_labels_set:
#         cluster_mask[clusters == c] = True
#     aux_plot(~cluster_mask)
#     plot_helper(~cluster_mask, (30, 50), (-10, 10), False, threshold)

In [ ]:
# Z_ward[-1, 2]

In [ ]:
# np.savetxt('data_2d_tsne.txt', data_2d_tsne, fmt = "%f")